# Anlamsal Çekirdek

Bu kod örneğinde, temel bir ajan oluşturmak için [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Çerçevesini kullanacaksınız.

Bu örneğin amacı, farklı ajan desenlerini uygularken sonraki kod örneklerinde kullanacağımız adımları size göstermektir.


## Gerekli Python Paketlerini İçe Aktar


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## İstemciyi Oluşturma

Bu örnekte, LLM'e erişim için [GitHub Modelleri](https://aka.ms/ai-agents-beginners/github-models) kullanılacaktır.

`ai_model_id` olarak `gpt-4o-mini` tanımlanmıştır. Farklı sonuçları görmek için GitHub Modelleri pazarında bulunan başka bir modeli deneyerek değiştirebilirsiniz.

GitHub Modelleri için `base_url` ile kullanılan `Azure Inference SDK`'yı kullanabilmemiz için, Semantic Kernel içinde `OpenAIChatCompletion` bağlayıcısını kullanacağız. Ayrıca, diğer model sağlayıcıları için Semantic Kernel'i kullanmak üzere başka [mevcut bağlayıcılar](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) da bulunmaktadır.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Ajanı Oluşturma

Aşağıda `TravelAgent` adında bir Ajan oluşturuyoruz.

Bu örnek için oldukça basit talimatlar kullanıyoruz. Bu talimatları değiştirerek ajanın nasıl farklı tepkiler verdiğini görebilirsiniz.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Ajanı Çalıştırma

Artık `ChatHistoryAgentThread` türünde bir thread tanımlayarak Ajanı çalıştırabiliriz. Gerekli olan sistem mesajları, ajanı invoke_stream'in `messages` anahtar argümanına sağlanır.

Bunlar tanımlandıktan sonra, kullanıcının ajana göndereceği mesaj olan `user_inputs` oluşturulur. Bu örnekte, bu mesajı `Bana güneşli bir tatil planla` olarak ayarladık.

Bu mesajı değiştirerek ajanın farklı şekilde nasıl yanıt verdiğini görmekten çekinmeyin.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
